In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
!pip3 install pmdarima --break-system-packages

In [4]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math


**Load and Preprocess Data**

In [5]:
import os
os.getcwd()

'/Users/fahadzuhair/Downloads/project_server_code/templates'

In [6]:
filename = '/content/drive/MyDrive/stock_price_prediction/filtered_stock_data.csv'
# filename = '/kaggle/input/filtered-stock-data/filtered_stock_data.csv'




In [7]:
df=pd.read_csv(filename)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/stock_price_prediction/filtered_stock_data.csv'

In [ ]:
df

,date,tiker,open,high,low,close,volume
0,2021-05-06,1010,24.80,24.82,24.32,24.42,967687
1,2021-05-05,1010,25.10,25.10,24.76,24.80,745371
2,2021-05-04,1010,25.50,25.60,24.76,24.90,1935432
3,2021-05-03,1010,25.90,25.90,25.45,25.45,745641
4,2021-05-02,1010,26.20,26.40,25.90,25.90,1087119
...,...,...,...,...,...,...,...
493302,1994-02-01,TASI,1752.26,1749.70,1749.70,1749.70,176595
493303,1994-01-31,TASI,1752.59,1752.26,1752.26,1752.26,212105
493304,1994-01-30,TASI,1750.91,1752.59,1752.59,1752.59,180425
493305,1994-01-29,TASI,1751.71,1750.91,1750.91,1750.91,204831


In [ ]:
#convert datetime from Object to datetime formate
df['date'] = pd.to_datetime(df['date'])

In [ ]:
#set datetime as dataset index
df.set_index('date', inplace=True)

In [ ]:
#Check if there are outlier values by looking at the standard deviation and mean.
df.describe()


,open,high,low,close,volume
count,493307.000000,493307.000000,493307.000000,493307.000000,4.933070e+05
mean,117.974713,119.069481,116.843502,118.010262,3.152230e+06
std,815.562207,821.779284,808.093757,815.672372,2.257261e+07
min,1.180000,1.180000,1.150000,1.150000,1.000000e+00
25%,12.420000,12.560000,12.300000,12.430000,9.911200e+04
50%,23.130000,23.450000,22.850000,23.160000,3.694010e+05
75%,43.200000,43.900000,42.630000,43.220000,1.224636e+06
max,20634.860000,20966.580000,20368.530000,20634.860000,1.064570e+09


In [ ]:

df['tiker'] = df['tiker'].astype(str)


In [ ]:
stock_Symbol_List = df['tiker'].unique().tolist()
print(stock_Symbol_List)


['1010', '1020', '1030', '1050', '1060', '1080', '1120', '1140', '2010', '2020', '2030', '2040', '2050', '2060', '2070', '2080', '2090', '2100', '2110', '2120', '2130', '2140', '2150', '2160', '2170', '2180', '2190', '2200', '2210', '2220', '2230', '2240', '2250', '2270', '2280', '2290', '2300', '2310', '2320', '3010', '3020', '3030', '3040', '3050', '3060', '3080', '3090', '4010', '4020', '4030', '4040', '4050', '4061', '4070', '4080', '4090', '4100', '4110', '4130', '4140', '4150', '4170', '4180', '4190', '4200', '4210', '4220', '4230', '4240', '5110', '6010', '6020', '6040', '6050', '6060', '6070', '6090', '7010', '7020', '8010', 'TASI']


In [ ]:
df=df[["close","tiker"]]

In [ ]:
df

,close,tiker
date,,
2021-05-06,24.42,1010
2021-05-05,24.80,1010
2021-05-04,24.90,1010
2021-05-03,25.45,1010
2021-05-02,25.90,1010
...,...,...
1994-02-01,1749.70,TASI
1994-01-31,1752.26,TASI
1994-01-30,1752.59,TASI


In [ ]:

df.sort_index(ascending=True,inplace=True)


lets we show some stock datetime index

As we see above ⏮ the dataset you provided, the dates are not continuous, and there are missing dates. This irregularity in the frequency of the data can make it challenging to directly apply ARIMA modeling, which typically assumes a regular frequency.

Note: In the following section, we will work with the "**ticker**" column as the number of stocks in our dataset.

**Analyse and Visualize**

In [ ]:
df.index=pd.to_datetime(df.index)

In [ ]:
yearly_counts = df.groupby(df.index.year).size()

In [ ]:
yearly_counts

date
1993     8490
1994    10605
1995    11676
1996    12130
1997    13431
1998    12836
1999    13157
2000    14011
2001    14337
2002    15886
2003    17825
2004    19784
2005    21027
2006    20033
2007    19839
2008    20167
2009    20124
2010    20169
2011    20088
2012    20328
2013    19906
2014    19996
2015    19982
2016    19819
2017    19841
2018    20071
2019    20222
2020    20252
2021     7275
dtype: int64

Visualize the stock’s daily closing price.

In [ ]:
filtered_df=df.copy()

In [ ]:
# stock_Symbol_List=['1010']

In [ ]:
# Plot closing prices for multiple stocks
# def Plot_closing_prices(stock_symbol,New_df=df):
#     plt.figure(figsize=(8, 4))
#     plt.plot(New_df[New_df['tiker'] == stock_symbol]["close"])
#     plt.ylabel("Price")
#     plt.xlabel("Date")
#     plt.title(str(i) + " Closing Stock Price")
#     plt.show()


# for i in stock_Symbol_List:
#   Plot_closing_prices(i)



In [ ]:
from statsmodels.tools.sm_exceptions import ValueWarning
warnings.simplefilter('ignore', ValueWarning)

In [ ]:
def resample_monthly_mean(df_data):

   # Resample to monthly frequency
   monthly = df_data['close'].resample('M').mean()

   # Forward fill to fill in missing values
   monthly_filled = monthly.ffill()

   return monthly_filled


In [ ]:
from matplotlib.dates import YearLocator

In [ ]:
filtered_df.isnull().sum()

close    0
tiker    0
dtype: int64

In [ ]:
stock_resample={}
#y = stock['Close'].resample('M').mean()
for i in stock_Symbol_List:
  df_monthly_mean=resample_monthly_mean(filtered_df[df['tiker']==i][['close']])
  stock_resample[i]=df_monthly_mean
  # df_monthly_mean.plot(figsize=(15, 8))
  # plt.title('Stock  ' + str(i)+  '  resample to year')
  # plt.show()


In [ ]:
# filtered_df[filtered_df['tiker']=='2090'][['close']]

In [ ]:
# Convert the dictionary to a DataFrame
df_monthly = pd.concat(stock_resample.values(), keys=stock_resample.keys())


In [ ]:
df_monthly.isnull().sum()

0

Some distinguishable patterns emerge when we plot the data. The time-series exhibits a seasonal pattern.

We can further visualize our data using a technique called time-series decomposition. This method enables us to break down our time series into three distinct components: trend, seasonality, and noise.

In [ ]:
df_monthly

      date      
1010  1993-01-31        7.247143
      1993-02-28        7.243000
      1993-03-31        7.365000
      1993-04-30        7.306000
      1993-05-31        6.833500
                        ...     
TASI  2021-01-31     8808.448571
      2021-02-28     8938.093500
      2021-03-31     9497.363913
      2021-04-30    10068.667619
      2021-05-31    10294.864000
Name: close, Length: 24164, dtype: float64

In [ ]:
import statsmodels.api as sm

In [ ]:
# from pylab import rcParams
# def plot_decompostion(data,stock):
#   rcParams['figure.figsize'] = 18, 8
#   decomposition = sm.tsa.seasonal_decompose(data, model='additive')
#   fig = decomposition.plot()
#   plt.title(str(stock))
#   plt.show()

# for i in stock_Symbol_List:
#         plot_decompostion(df_monthly[i],i)

The plot above clearly shows the instability of the Close price, as well as its noticeable seasonality.

**Time Series Forecasting with ARIMA**



In this analysis, we'll explore one of the most widely used methods for time-series forecasting: ARIMA, which stands for Autoregressive Integrated Moving Average.

Parameter Selection for the ARIMA Time Series Model

ARIMA models are denoted with the notation ARIMA(p, d, q). These three parameters are crucial for capturing seasonality, trend, and noise in the data.

In [ ]:
df_monthly.keys()

MultiIndex([('1010', '1993-01-31'),
            ('1010', '1993-02-28'),
            ('1010', '1993-03-31'),
            ('1010', '1993-04-30'),
            ('1010', '1993-05-31'),
            ('1010', '1993-06-30'),
            ('1010', '1993-07-31'),
            ('1010', '1993-08-31'),
            ('1010', '1993-09-30'),
            ('1010', '1993-10-31'),
            ...
            ('TASI', '2020-08-31'),
            ('TASI', '2020-09-30'),
            ('TASI', '2020-10-31'),
            ('TASI', '2020-11-30'),
            ('TASI', '2020-12-31'),
            ('TASI', '2021-01-31'),
            ('TASI', '2021-02-28'),
            ('TASI', '2021-03-31'),
            ('TASI', '2021-04-30'),
            ('TASI', '2021-05-31')],
           names=[None, 'date'], length=24164)

In [ ]:
import itertools


In [ ]:
p = d = q = range(0, 2)
param_combinations = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:


def find_optimal_sarimax_params(data_monthly_mean):
    param_list = []
    param_seasonal_list = []
    results_aic_list = []
    for param in param_combinations:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(data_monthly_mean,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)

                results = mod.fit()

                # print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))

                param_list.append(param)
                param_seasonal_list.append(param_seasonal)
                results_aic_list.append(results.aic)
            except:
                continue

    return param_list, param_seasonal_list, results_aic_list


In [ ]:
def get_best_aic(results_list):
  best_aic = results_list[0]
  for aic in results_list[1:]:
      if aic < best_aic:
         best_aic = aic
  best_index = results_list.index(best_aic)
  return best_index


In [ ]:
def train_sarimax_model(data_monthly_mean):
    param_list, param_seasonal_list, results_aic_list=find_optimal_sarimax_params(data_monthly_mean)
    prem=get_best_aic(results_aic_list)
    best_order = param_list[prem]

    best_seasonal_order = param_seasonal_list[get_best_aic(results_aic_list)]

    best_model = sm.tsa.statespace.SARIMAX(data_monthly_mean,
                                       order=best_order,
                                       seasonal_order=best_seasonal_order,
                                       enforce_stationarity=False,
                                       enforce_invertibility=False)

    best_results = best_model.fit()
    return best_results





In [ ]:
models_path='/content/drive/MyDrive/ARIMA_model/arima_stock_models.pkl'

In [ ]:
import pickle
import joblib
# Function to load existing models from a pickle file
def load_models(filepath):
    try:
        with open(filepath, "rb") as file:
            return  joblib.load(file)
    except (FileNotFoundError, EOFError):
        return {}

In [ ]:
models=load_models(models_path)

In [ ]:
last_trained_len=len(models.keys())

**Here, we will start training by setting the session size. Note: The session size refers to the number of stocks that we want to train during this session.**

In [ ]:
session_size=2
end_train_stock_=min(last_trained_len+session_size,len(stock_Symbol_List))

In [ ]:
models={}
for stock_symbol in stock_Symbol_List[last_trained_len:end_train_stock_]:
  model=train_sarimax_model(df_monthly[stock_symbol])
  models[stock_symbol]=model
  print(f"Results for stock: {stock_symbol}")
  # print(model.summary())

Results for stock: 1050
Results for stock: 1060


**Validating Forecasts**


To assess the accuracy of our forecasts, we compare the predicted Close prices to the actual close prices of the time series. We focus our evaluation on forecasts starting from 2018-12-31 to the end of the available data.

In [ ]:
models.keys()

dict_keys(['1050', '1060'])

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

forecast_start_date = pd.to_datetime('2018-12-31')

def evaluate_forecast_accuracy(current_models_stock):
    for stock_symbol in current_models_stock.keys():
        # Get predictions from the start date to the end of the series
        pred = current_models_stock[stock_symbol].get_prediction(start=forecast_start_date, dynamic=False)
        pred_ci = pred.conf_int()
        y_forecasted = pred.predicted_mean
        y_truth = df_monthly[stock_symbol][forecast_start_date:]

        # Compute the mean square error
        mse = ((y_forecasted - y_truth) ** 2).mean()
        print(f'The Mean Squared Error of our forecasts for {stock_symbol} is {round(mse, 2)}')
        print(f'The Root Mean Squared Error of our forecasts for {stock_symbol} is {round(np.sqrt(mse), 2)}')
        print('\n\n')

        # Plot
        # plt.figure(figsize=(14, 7))
        # plt.plot(df_monthly[stock_symbol]['2012':], label='Original', color='blue')
        # plt.plot(y_forecasted, label='Forecast', alpha=0.7, color='orange')
        # plt.fill_between(pred_ci.index,
        #                  pred_ci.iloc[:, 0],
        #                  pred_ci.iloc[:, 1], color='k', alpha=0.2)
        # plt.xlabel('Date')
        # plt.ylabel(f'{stock_symbol} Close Price')
        # plt.title(f'Original vs Forecasted Close Price for {stock_symbol}')
        # plt.legend()
        # plt.show()

evaluate_forecast_accuracy(models)



The Mean Squared Error of our forecasts for 1050 is 6.32
The Root Mean Squared Error of our forecasts for 1050 is 2.51



The Mean Squared Error of our forecasts for 1060 is 5.47
The Root Mean Squared Error of our forecasts for 1060 is 2.34





The line plot displays the original values alongside the rolling forecast predictions. In general, our forecasts closely match the actual values, demonstrating a consistent upward trend that begins from the start of the year

**forecasting the next n months**

In [ ]:


def forecasting_future_month(stock_models, months):
    for stock_symbol in stock_models.keys():
        pred_uc = stock_models[stock_symbol].get_forecast(steps=months)
        pred_ci = pred_uc.conf_int()
        print(f'Close price forecasting values for stock:{stock_symbol} are\n', pred_uc.predicted_mean)
        # ax = df_monthly[stock_symbol].plot(label='Observed', figsize=(14, 7))
        # pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
        # ax.fill_between(pred_ci.index,
        #                 pred_ci.iloc[:, 0],
        #                 pred_ci.iloc[:, 1], color='k', alpha=.25)
        # ax.set_xlabel('Date')
        # ax.set_ylabel('Close Price')
        # ax.set_title(f'Forecast for Stock Symbol {stock_symbol}')
        # plt.legend()
        # plt.show()
forecasting_future_month(models, 9)


Close price forecasting values for stock:1050 are
 2021-06-30    36.532546
2021-07-31    36.487514
2021-08-31    36.616779
2021-09-30    36.486196
2021-10-31    35.924960
2021-11-30    36.564045
2021-12-31    36.543614
2022-01-31    36.862177
2022-02-28    36.625794
Freq: M, Name: predicted_mean, dtype: float64
Close price forecasting values for stock:1060 are
 2021-06-30    29.561290
2021-07-31    29.607733
2021-08-31    30.014670
2021-09-30    29.953410
2021-10-31    29.465829
2021-11-30    30.020086
2021-12-31    29.996266
2022-01-31    30.351336
2022-02-28    30.354897
Freq: M, Name: predicted_mean, dtype: float64


**Now we will append the current model's training with the last.**

In [ ]:
import pickle
import joblib
# Function to load existing models from a pickle file
def load_models(filepath):
    try:
        with open(filepath, "rb") as file:
            return  joblib.load(file)
    except (FileNotFoundError, EOFError):
        return {}  # If the file is not found or empty, return an empty dict


In [ ]:
def save_models(models, filepath):
    existing_models = load_models(filepath)  # Load any existing models
    existing_models.update(models)  # Update with new models
    with open(filepath, "wb") as file:
        # pickle.dump(existing_models, file)
        joblib.dump(existing_models, file, compress=3)
         # Save the updated model dictionary

In [ ]:
save_models(models,models_path)

# Now we can Load all stock files for evaluate accuracy and forecasting,



In [ ]:
saved_model_path='/content/drive/MyDrive/ARIMA_model/arima_stock_models.pkl'

In [ ]:

# Load the saved models from the file
with open(saved_model_path, "rb") as file:
    last_all_model_train = joblib.load(file)


In [ ]:
last_all_model_train.keys()

dict_keys(['1010', '1020', '1030', '1050', '1060'])

In [ ]:
evaluate_forecast_accuracy(last_all_model_train)

The Mean Squared Error of our forecasts for 1010 is 2.71
The Root Mean Squared Error of our forecasts for 1010 is 1.65



The Mean Squared Error of our forecasts for 1020 is 0.88
The Root Mean Squared Error of our forecasts for 1020 is 0.94



The Mean Squared Error of our forecasts for 1030 is 1.12
The Root Mean Squared Error of our forecasts for 1030 is 1.06



The Mean Squared Error of our forecasts for 1050 is 6.32
The Root Mean Squared Error of our forecasts for 1050 is 2.51



The Mean Squared Error of our forecasts for 1060 is 5.47
The Root Mean Squared Error of our forecasts for 1060 is 2.34





In [ ]:
input_stock = input('Please enter the stock symbol: ')
for stock_symbol in last_all_model_train.keys():
    if stock_symbol == input_stock:
        pred_uc = models[stock_symbol].get_forecast(steps=10)#steps=10 ,meaning 10 months future forecasting
        print('Close price forecasting values:', pred_uc.tvalues)


Please enter the stock symbol: 1020
Close price forecasting values: 2021-06-30    9.435003
2021-07-31    5.987253
2021-08-31    4.582000
2021-09-30    3.746155
2021-10-31    3.173220
2021-11-30    2.781488
2021-12-31    2.547639
2022-01-31    2.369804
2022-02-28    2.242441
2022-03-31    2.084825
Freq: M, Name: tvalues, dtype: float64
